# 1. Install Library

In [ ]:
pip install pyspark

In [44]:
pip install pandas-ml

Could not build wheels for pandas-ml, since package 'wheel' is not installed.
Could not build wheels for pandas, since package 'wheel' is not installed.
Could not build wheels for enum34, since package 'wheel' is not installed.
Could not build wheels for numpy, since package 'wheel' is not installed.
Could not build wheels for pytz, since package 'wheel' is not installed.
Could not build wheels for python-dateutil, since package 'wheel' is not installed.
Could not build wheels for six, since package 'wheel' is not installed.
Note: you may need to restart the kernel to use updated packages.


In [16]:
pip install pandas

Could not build wheels for pandas, since package 'wheel' is not installed.
Could not build wheels for numpy, since package 'wheel' is not installed.
Could not build wheels for pytz, since package 'wheel' is not installed.
Could not build wheels for python-dateutil, since package 'wheel' is not installed.
Could not build wheels for six, since package 'wheel' is not installed.
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install numpy

Could not build wheels for numpy, since package 'wheel' is not installed.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install liniear-regression

# 2. Import Spark SQL, Machine Learning & Membuat Session

In [3]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark import SparkContext, SparkConf

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))

#Buat Session
appName = "Regresi Spark"
spark = SparkSession \
    .builder \
    .appName(appName) \
    .config("spark.some.config.ooption", "some-value") \
    .getOrCreate()

# 3. Membuat Data Frame dari File

In [4]:
flightSchema = StructType([
    StructField("DayOfMonth", IntegerType(), False),
    StructField("DayOfWeek", IntegerType(), False),
    StructField("Carrier", StringType(), False),
    StructField("OriginalAirportID", IntegerType(), False),
    StructField("DestAirportID", IntegerType(), False),
    StructField("DepDelay", IntegerType(), False),
    StructField("ArrDelay", IntegerType(), False)
])
#Baca data file
flightDataFrame = spark.read.csv('/root/materi/mtk-stats/Regresi Linier/dataset/flights.csv', schema=flightSchema, header=True)
flightDataFrame.show(5)

+----------+---------+-------+-----------------+-------------+--------+--------+
|DayOfMonth|DayOfWeek|Carrier|OriginalAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+-----------------+-------------+--------+--------+
|        19|        5|     DL|            11433|        13303|      -3|       1|
|        19|        5|     DL|            14869|        12478|       0|      -8|
|        19|        5|     DL|            14057|        14869|      -4|     -15|
|        19|        5|     DL|            15016|        11433|      28|      24|
|        19|        5|     DL|            11193|        12892|      -6|     -11|
+----------+---------+-------+-----------------+-------------+--------+--------+
only showing top 5 rows



# 4. Mempersiapkan Data

In [5]:
data = flightDataFrame.select("DayOfMonth", "DayOfWeek", "OriginalAirportID", "DestAirportID", "DepDelay", "ArrDelay")
data.show(5)

+----------+---------+-----------------+-------------+--------+--------+
|DayOfMonth|DayOfWeek|OriginalAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-----------------+-------------+--------+--------+
|        19|        5|            11433|        13303|      -3|       1|
|        19|        5|            14869|        12478|       0|      -8|
|        19|        5|            14057|        14869|      -4|     -15|
|        19|        5|            15016|        11433|      28|      24|
|        19|        5|            11193|        12892|      -6|     -11|
+----------+---------+-----------------+-------------+--------+--------+
only showing top 5 rows



# 5. Membagi data training & data testing

In [6]:
#70% Data Training dan 30% Data Testing
dataTerpisahkan = data.randomSplit([0.7, 0.3])
trainingData = dataTerpisahkan[0]
testingData = dataTerpisahkan[1]
train_rows = trainingData.count()
test_rows = testingData.count()
print("Training Rows : ", train_rows, " Testing Rows : ", test_rows)

Training Rows :  1891056  Testing Rows :  811162


# 6. Menyiapkan data Training

In [10]:
assembler = VectorAssembler(inputCols = [
    "DayOfMonth", "DayOfWeek", "OriginalAirportID", "DestAirportID", "DepDelay"], outputCol="features")
trainingDataFinal = assembler.transform(trainingData).select(col("features"), (col("ArrDelay").cast("Int").alias("label")))
trainingDataFinal.show(truncate=False, n =3)

+------------------------------+-----+
|features                      |label|
+------------------------------+-----+
|[1.0,1.0,10140.0,10397.0,-4.0]|-11  |
|[1.0,1.0,10140.0,10397.0,0.0] |-12  |
|[1.0,1.0,10140.0,10397.0,0.0] |-9   |
+------------------------------+-----+
only showing top 3 rows



# 7. Training Algoritma Model Regresi

In [11]:
algoritma = LinearRegression(
    labelCol="label", featuresCol="features", maxIter=10, regParam=0.3)
model = algoritma.fit(trainingDataFinal)
print("Model Algoritma Regresi selesai di Training")

Model Algoritma Regresi selesai di Training


# 8. Mempersiapkan Data Testing

In [12]:
testingDataFinal = assembler.transform(testingData).select(
    col("features"), (col("ArrDelay")).cast("Int").alias("trueLabel"))
testingDataFinal.show(truncate=False, n=2)

+------------------------------+---------+
|features                      |trueLabel|
+------------------------------+---------+
|[1.0,1.0,10140.0,10397.0,-2.0]|-18      |
|[1.0,1.0,10140.0,10397.0,-2.0]|-17      |
+------------------------------+---------+
only showing top 2 rows



# 9. Memprediksi hasil regresi dengan model yang sudah di training

In [13]:
prediksiMentah = model.transform(testingDataFinal)
prediksiFinal = prediksiMentah.select("features", "prediction", "trueLabel")
prediksiFinal.show(3)

+--------------------+-------------------+---------+
|            features|         prediction|trueLabel|
+--------------------+-------------------+---------+
|[1.0,1.0,10140.0,...| -5.538298828000444|      -18|
|[1.0,1.0,10140.0,...| -5.538298828000444|      -17|
|[1.0,1.0,10140.0,...|0.34627990534253206|        4|
+--------------------+-------------------+---------+
only showing top 3 rows



# 10. Mengukur Akurasi model Regresi

In [14]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(
    labelCol="trueLabel", predictionCol= "prediction", metricName="rmse")
rmse = evaluator.evaluate(prediksiFinal)
print("Root Mean Square Error (RMSE) : ",rmse)

Root Mean Square Error (RMSE) :  13.219716358534328


# 11. Memprediksi satu data Penerbangan

In [24]:
baris1List = testingData.take(1)
baris1DataFrame = sc.parallelize(baris1List).toDF() #Mengubah list ke RDD ke dataFrame

testingBaris1= assembler.transform(
    baris1DataFrame).select(col("features"), (col("ArrDelay")).cast("Int").alias("trueLabel"))
prediksiMentah = model.transform(testingBaris1)
prediksiFinal = prediksiMentah.select("features", "prediction", "trueLabel")
prediksiFinal.show()

+--------------------+------------------+---------+
|            features|        prediction|trueLabel|
+--------------------+------------------+---------+
|[1.0,1.0,10140.0,...|-5.538298828000444|      -18|
+--------------------+------------------+---------+

